Predicting every 4th letter by reading 3 letter

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

/home/kaju/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kaju/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/kaju/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH='data/nietzsche/'

In [3]:
text= open(f'{PATH}nietzsche.txt').read()

In [4]:
text[:300]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, ha'

In [11]:
chars1 = set(text)
#chars1

In [32]:
chars2 = list(chars1)
#chars2

In [19]:
chars3 = sorted(chars2)
len(chars3)

84

In [28]:
chars3.remove('0')

In [34]:
chars3.insert(0, "\0")


In [77]:
vocab_size = len(chars3) + 1

In [36]:
''.join(chars3[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [38]:
char_indices = {c: i for i, c in enumerate(chars3)}
idx_indices = {i: c for i, c in enumerate(chars3)}

In [39]:
idx = [char_indices[c] for c in text]

In [44]:
''.join(idx_indices[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [69]:
cs=3
c1 = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2 = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3 = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4 = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

In [70]:
len(c2)

200297

In [71]:
x1 = np.stack(c1)
x2 = np.stack(c2)
x3 = np.stack(c3)
x1

array([40, 30, 29, ..., 72, 59, 67])

In [75]:
x1.shape, y.shape

((200297,), (200297,))

In [74]:
y = np.stack(c4)
#np.append(y,0,axis=1)

In [55]:
n_hidden = 256
n_fac = 42

In [82]:
class Charmodel(nn.Module):
    def __init__(self,vocab_size,n_factor):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_factor)
        self.l_in = nn.Linear(n_factor, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))
        

In [83]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3],axis=1), y, bs=512)
m = Charmodel(vocab_size, n_fac).cuda()

In [84]:
it = iter(md.trn_dl)
*xs, yt = next(it)
t = m(*V(xs))
opt = optim.Adam(m.parameters(), 1e-2)

In [85]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      2.089107   1.014216  



[array([1.01422])]

In [86]:
set_lrs(opt, 0.001)

In [87]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                              
    0      1.829891   0.472934  



[array([0.47293])]

In [93]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars3[i]

In [94]:
#get_next('y. ')
get_next('y. ')

'T'

In [95]:
get_next('ppl')

'e'

In [96]:
get_next('dse')

'l'